In [18]:
#reading the data from the dataset and converting to lowercase, converting list of reviews
import nltk


In [19]:
file = 'dataset2.csv'
#with open(file, 'rb') as fopen:
        #corpus = fopen.read()
        #print(corpus.decode('latin-1'))
corpus = open('dataset2.csv').read()
corpus = corpus.lower()
docs = corpus.split('\n')


#creatinig a list of the only POS tags that we are interested in
words = nltk.tokenize.word_tokenize(docs[0])
allowed_tags = ['VBP', 'VB', 'VBG', 'JJ', 'NN', 'RB']

In [21]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from string import punctuation as punc

pdocs = []
for d in docs:
    words = nltk.tokenize.word_tokenize(d)
    words_tags = nltk.pos_tag(words)
    for w, t in words_tags:
        if w in punc:
            words.remove(w)
        elif w in ENGLISH_STOP_WORDS:
            words.remove(w)
        elif t not in allowed_tags:
            words.remove(w)
    pd = ' '.join(words)
    pdocs.append(pd)



ModuleNotFoundError: No module named 'sklearn.feature_extraction.stop_words'

In [ ]:
#Structuring our input documents with CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
matrix_X = vec.fit_transform(pdocs)



In [ ]:
#LDA (Topic modeling)
features = vec.get_feature_names()
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5)
lda.fit(matrix_X)

In [ ]:
#Sentiment Analysis
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

for tid, topics in enumerate(lda.components_):
    print('topic ID: ', tid)
    top_words = [features[i] for i in topics.argsort()[:-10:-1]]
    print(top_words)
    score = 0
    for w in top_words:
        senti_synset = swn.senti_synset(wn.synsets(w)[0].name())
        score += senti_synset.pos_score() - senti_synset.neg_score()
    print('Sentiment Score: ', score)

### Classification 

In [22]:
#tokenizing and covertint to lower-case
from nltk.corpus import wordnet
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from string import punctuation as punc
import nltk.tokenize
raw = open("E:\\dataset.txt").read()
raw = raw.lower()
docs = nltk.tokenize.sent_tokenize(raw)
docs = docs[0].split('\n')

#pre-processing punctuations
for d in docs:
    for ch in d:
        if ch in punc:
            d = d.replace(ch, '')

#removing stopwords, stemming words
ps = PorterStemmer()
for d in docs:
    for token in nltk.tokenize.word_tokenize(d):
        if token in ENGLISH_STOP_WORDS:
            d = d.replace(token, '')
        d = d.replace(token, ps.stem(token))

# asking for the test document from the user through direct input
for i in range(len(docs)):
    print('D' + str(i) + ": " + docs[i])
test = input("Enter your text: ")
docs.append(test + ":")

#separating input documents from labels, stripping off the unwanted spaces
X, y = [], []
for d in docs:
    X.append(d[:d.index(":")].lstrip().rstrip())
    y.append(d[d.index(":")+1:].lstrip().rstrip())

#Vectorizing with Tfidf vectorizer
vectorizer = TfidfVectorizer()
vec = vectorizer.fit_transform(X)
print(vec.toarray)

#training KNN Classifier
clf = sklearn.neighbors.KNeighborsClassifier(1)
clf.fit(vec[:6], y[:6])
print('Label: ', clf.predict(vec[6]))

#sentiment analysis
test_tokens = test.split(' ')
good = wordnet.synsets('good')
bad = wordnet.synsets('evil')
score_pos = score_neg = 0

for token in test_tokens:
    t = wordnet.synsets(token)
    if len(t) > 0:
        sim_good = wordnet.wup_similarity(good[0], t[0])
        sim_bad = wordnet.wup_similarity(bad[0], t[0])
        if(sim_good is not None):
            score_pos = score_pos + sim_good
        if(sim_bad is not None):
            score_neg = score_neg + sim_bad

if(score_neg - score_pos > 0.1):
    print('Subjective statement, Negative opinion of strength: %.2f' % score_neg)
elif(score_pos - score_neg > 0.1):
    print('Subjective statement, Positive opinion of strength: %.2f' % score_pos)
else:
    print('Objective statement, No opinion showed')

#nearest documents
nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=2)
nbrs.fit(vec[:6])
closest_docs = nbrs.kneighbors(vec[6])
print('Recommended readings are documents with IDs ', closest_docs[1])
print('having distances ', closest_docs[0])


ModuleNotFoundError: No module named 'sklearn.feature_extraction.stop_words'